## etri dataloader 제작

In [1]:
import argparse
import json
import os
import sys

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning, NumbaWarning
import warnings
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaWarning)

import numpy as np
import torch
import yaml
from det3d.datasets import build_dataset
from det3d.models import build_detector
from det3d.torchie import Config
from det3d.torchie.apis import (
    build_optimizer,
    get_root_logger,
    init_dist,
    set_random_seed,
    train_detector,
)
import torch.distributed as dist
import subprocess
# 주피터 노트북 경로설정
os.chdir('../')
def parse_args():
    parser = argparse.ArgumentParser(description="Train a detector")
    parser.add_argument("--config", default="configs/etriInfra/pp/etriInfra_centerpoint_pp_02voxel_two_pfn_10sweep.py", help="train config file path")
    parser.add_argument("--work_dir", help="the dir to save logs and models")
    parser.add_argument("--resume_from", help="the checkpoint file to resume from")
    parser.add_argument(
        "--validate",
        action="store_true",
        help="whether to evaluate the checkpoint during training",
    )
    parser.add_argument(
        "--gpus",
        type=int,
        default=1,
        help="number of gpus to use " "(only applicable to non-distributed training)",
    )
    parser.add_argument("--seed", type=int, default=None, help="random seed")
    parser.add_argument(
        "--launcher",
        choices=["pytorch", "slurm"],
        default="pytorch",
        help="job launcher",
    )
    parser.add_argument("--local_rank", type=int, default=0)
    parser.add_argument(
        "--autoscale-lr",
        action="store_true",
        help="automatically scale lr with the number of gpus",
    )
    args = parser.parse_args(args=[])
    if "LOCAL_RANK" not in os.environ:
        os.environ["LOCAL_RANK"] = str(args.local_rank)

    return args
args = parse_args()
cfg = Config.fromfile(args.config)

no apex
No Tensorflow
Deformable Convolution not built!
No APEX!


In [2]:
cfg.local_rank = args.local_rank
distributed = False
logger = get_root_logger(cfg.log_level)
logger.info("Distributed training: {}".format(distributed))
logger.info(f"torch.backends.cudnn.benchmark: {torch.backends.cudnn.benchmark}")

2024-04-08 15:53:03,418 - INFO - Distributed training: False
2024-04-08 15:53:03,419 - INFO - torch.backends.cudnn.benchmark: False


In [3]:
cfg.data.train

{'type': 'etrInfraDataset',
 'root_path': 'data/etri3Dobj_infra_edge',
 'info_path': 'data/etri3Dobj_infra_edge/infos_train.pkl',
 'ann_file': 'data/etri3Dobj_infra_edge/infos_train.pkl',
 'nsweeps': 1,
 'class_names': ['car',
  'personal_mobility',
  'truck',
  'construction_vehicle',
  'bus',
  'ground_animal',
  'motorcycle',
  'bicycle',
  'pedestrian'],
 'pipeline': [{'type': 'LoadPointCloudFromFile', 'dataset': 'etrInfraDataset'},
  {'type': 'LoadPointCloudAnnotations', 'with_bbox': True},
  {'type': 'Preprocess',
   'cfg': {'mode': 'train',
    'shuffle_points': True,
    'global_rot_noise': [-0.3925, 0.3925],
    'global_scale_noise': [0.95, 1.05],
    'db_sampler': {'type': 'GT-AUG',
     'enable': False,
     'db_info_path': 'data/etri3Dobj_infra_edge/db_meta_withoutvelo.pkl',
     'sample_groups': [{'car': 1},
      {'personal_mobility': 3},
      {'truck': 3},
      {'construction_vehicle': 3},
      {'bus': 3},
      {'ground_animal': 3},
      {'motorcycle': 6},
      {'b

### build_dataset

In [4]:
import pickle
from det3d.datasets.registry import DATASETS

In [5]:
args = cfg.data.train.copy()
obj_type = args.pop("type")

In [6]:
for k, v in args.items():
    print(f"{k}: {v}")

root_path: data/etri3Dobj_infra_edge
info_path: data/etri3Dobj_infra_edge/infos_train.pkl
ann_file: data/etri3Dobj_infra_edge/infos_train.pkl
nsweeps: 1
class_names: ['car', 'personal_mobility', 'truck', 'construction_vehicle', 'bus', 'ground_animal', 'motorcycle', 'bicycle', 'pedestrian']
pipeline: [{'type': 'LoadPointCloudFromFile', 'dataset': 'etrInfraDataset'}, {'type': 'LoadPointCloudAnnotations', 'with_bbox': True}, {'type': 'Preprocess', 'cfg': {'mode': 'train', 'shuffle_points': True, 'global_rot_noise': [-0.3925, 0.3925], 'global_scale_noise': [0.95, 1.05], 'db_sampler': {'type': 'GT-AUG', 'enable': False, 'db_info_path': 'data/etri3Dobj_infra_edge/db_meta_withoutvelo.pkl', 'sample_groups': [{'car': 1}, {'personal_mobility': 3}, {'truck': 3}, {'construction_vehicle': 3}, {'bus': 3}, {'ground_animal': 3}, {'motorcycle': 6}, {'bicycle': 6}, {'pedestrian': 10}], 'db_prep_steps': [{'filter_by_min_num_points': {'car': 5, 'personal_mobility': 5, 'truck': 5, 'construction_vehicle': 5

In [7]:
# def __init__(self, info_path, root_path, nsweeps=0, cfg=None, pipeline=None, class_names=None, 
#               test_mode=False, version="v1.0-trainval", load_interval=1, **kwargs,)

# parameter
info_path = args["info_path"]
root_path = args["root_path"]
nsweeps = args["nsweeps"]
cfg = None
pipeline = args["pipeline"]
class_names = args["class_names"]
test_mode=False
version = ""
load_interval = 1

_info_path = info_path
_class_names = class_names

In [8]:
info_path

'data/etri3Dobj_infra_edge/infos_train.pkl'

In [9]:
# 약 6분 소요
with open(info_path, "rb") as f:
    _nusc_infos_all = pickle.load(f)
print(len(_nusc_infos_all))

25280


In [10]:
_nusc_infos_all[0]

{'type': 'lidar',
 'points': array([[-3.15695594e-04,  1.50720686e-01,  4.02389988e-02],
        [ 1.38273943e-04,  1.58450231e-01,  4.23025340e-02],
        [ 5.89517702e-04,  1.46855190e-01,  3.92072275e-02],
        ...,
        [-1.78722490e-03,  1.38371289e-01, -3.98270525e-02],
        [-1.39010733e-03,  1.42220005e-01, -4.09333594e-02],
        [-9.94250178e-04,  1.49911433e-01, -4.31459732e-02]]),
 'annotations': {'boxes': array([[ 4.46990350e+00, -4.27913468e+00, -1.05122426e+00,
           2.21067278e+00,  4.30493938e+00,  1.50000000e+00,
           0.00000000e+00,  0.00000000e+00, -2.49582083e+00],
         [-1.55185063e+00, -1.12952562e+01, -8.54921520e-01,
           2.19621365e+00,  5.14712374e+00,  1.85010689e+00,
           0.00000000e+00,  0.00000000e+00, -2.32128791e+00],
         [ 1.69509220e+00, -1.25010433e+01, -9.90051065e-01,
           2.05839914e+00,  4.88977311e+00,  1.62805410e+00,
           0.00000000e+00,  0.00000000e+00, -2.32128791e+00],
         [ 7.68

In [11]:
from det3d.datasets.nuscenes.nusc_common import (
    general_to_detection,
    cls_attr_dist,
    _second_det_to_nusc_box,
    _lidar_nusc_box_to_global,
    eval_main
)
from det3d.datasets.pipelines import Compose
_num_point_features = 3 # NuScenesDataset.NumPointFeatures
_name_mapping = general_to_detection
eval_version = "detection_cvpr_2019"

In [12]:
_pipeline = Compose(pipeline)

2024-04-08 15:53:16,959 - INFO - {'car': 5, 'personal_mobility': 5, 'truck': 5, 'construction_vehicle': 5, 'bus': 5, 'ground_animal': 5, 'motorcycle': 5, 'bicycle': 5, 'pedestrian': 5}
2024-04-08 15:53:16,960 - INFO - [-1]


2024-04-08 15:53:18,277 - INFO - load 474176 car database infos
2024-04-08 15:53:18,277 - INFO - load 18209 truck database infos
2024-04-08 15:53:18,278 - INFO - load 10857 bus database infos
2024-04-08 15:53:18,278 - INFO - load 5070 motorcycle database infos
2024-04-08 15:53:18,278 - INFO - load 6526 construction_vehicle database infos
2024-04-08 15:53:18,279 - INFO - load 28750 pedestrian database infos
2024-04-08 15:53:18,279 - INFO - load 11105 personal_mobility database infos
2024-04-08 15:53:18,279 - INFO - load 554 bicycle database infos
2024-04-08 15:53:18,279 - INFO - load 298 ground_animal database infos
2024-04-08 15:53:18,755 - INFO - After filter database:
2024-04-08 15:53:18,755 - INFO - load 338483 car database infos
2024-04-08 15:53:18,755 - INFO - load 16574 truck database infos
2024-04-08 15:53:18,756 - INFO - load 10475 bus database infos
2024-04-08 15:53:18,756 - INFO - load 2724 motorcycle database infos
2024-04-08 15:53:18,756 - INFO - load 6469 construction_vehi

In [13]:
from pathlib import Path
idx = 0
info = _nusc_infos_all[idx]
_root_path = Path(root_path)
res = {
    "lidar": {
        "type": "lidar",
        "points": None,
        "nsweeps": 1,
        # "ground_plane": -gp[-1] if with_gp else None,
        "annotations": None,
    },
    "metadata": {
        "image_prefix": Path("data/etri3Dobj_infra_edge"),
        "num_point_features": _num_point_features,
    },
    "calib": None,
    "cam": {},
    "mode": "val" if test_mode else "train"
}

In [14]:
for i in pipeline:
    print(i['type'])

LoadPointCloudFromFile
LoadPointCloudAnnotations
Preprocess
Voxelization
AssignLabel
Reformat


In [15]:
res

{'lidar': {'type': 'lidar', 'points': None, 'nsweeps': 1, 'annotations': None},
 'metadata': {'image_prefix': PosixPath('data/etri3Dobj_infra_edge'),
  'num_point_features': 3},
 'calib': None,
 'cam': {},
 'mode': 'train'}

In [16]:
data, _ = _pipeline(res, info)

2024-04-08 15:10:44,812 - INFO - finding looplift candidates


In [17]:
data

{'metadata': {'image_prefix': PosixPath('data/etri3Dobj_infra_edge'),
  'num_point_features': 3},
 'points': array([[ 1.80217207e+00, -1.15589991e+01, -1.85644173e+00],
        [ 3.99044090e-02,  1.58184511e-01,  1.12018768e-02],
        [-1.53205290e+00,  5.65383232e+00, -1.46364571e+00],
        ...,
        [ 3.34649985e-01, -7.01380389e+00, -1.88555382e+00],
        [-1.82174969e+01,  1.00724989e+01, -1.46622924e+00],
        [ 2.25471001e-02,  1.60465385e-01,  4.32613576e-02]]),
 'voxels': array([[[ 1.80217207e+00, -1.15589991e+01, -1.85644173e+00],
         [ 1.91103041e+00, -1.15415006e+01, -1.85644173e+00],
         [ 1.87476464e+00, -1.15474465e+01, -1.85644173e+00],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],
 
        [[ 3.99044090e-02,  1.58184511e-01,  1.12018768e-02],
         [ 3.36002344e-02,  1.51082795e-01,  1.062